In [10]:
import json
from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from tqdm import tqdm
from langchain_community.vectorstores import FAISS


# 定义文件路径
file_path = r"C:\Users\xhx20\Desktop\Agent\datasets\TeleQnA.txt"

# 读取文件内容
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)


# 提取问题及相关信息，转换为 Document 格式
documents = []
for key, value in data.items():
    content = (
        f"Question: {value['question']}\n"
        f"Explanation: {value['explanation']}\n"
        f"Category: {value['category']}"
    )
    documents.append(Document(page_content=content, metadata={"id": key}))


print("start vector")

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(
        api_key="sk-nI4Wy7Ws2Tj2piRR6a65045dEcD7446c84BfF8BeD27a7d13",
        base_url="https://free.v36.cm/v1/",
    ),
    persist_directory=r"C:\Users\xhx20\Desktop\Agent\agent_web\vectorstore"  # 可选，设置持久化路径
)


ImportError: cannot import name 'field_validator' from 'pydantic' (c:\Users\xhx20\.conda\envs\Pytorch\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [7]:
#!/usr/bin/env python
"""Endpoint shows off available playground widgets."""
import base64
from json import dumps
from typing import Any, Dict, List, Tuple

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from langchain_community.document_loaders.parsers.pdf import PDFMinerParser
from langchain_core.document_loaders import Blob
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

from langserve import CustomUserType
from langserve.server import add_routes

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="Spin up a simple api server using Langchain's Runnable interfaces",
)


# Set all CORS enabled origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

# Example 1: Chat Widget
# This shows how to create a chat widget.


class ChatHistory(CustomUserType):
    chat_history: List[Tuple[str, str]] = Field(
        ...,
        examples=[[("a", "aa")]],
        extra={"widget": {"type": "chat", "input": "question", "output": "answer"}},
    )
    question: str


def _format_to_messages(input: ChatHistory) -> List[BaseMessage]:
    """Format the input to a list of messages."""
    history = input.chat_history
    user_input = input.question

    messages = []

    for human, ai in history:
        messages.append(HumanMessage(content=human))
        messages.append(AIMessage(content=ai))
    messages.append(HumanMessage(content=user_input))
    return messages


model = ChatOpenAI(
    api_key="sk-nI4Wy7Ws2Tj2piRR6a65045dEcD7446c84BfF8BeD27a7d13",
    base_url="https://free.v36.cm/v1/",
)
chat_model = RunnableParallel({"answer": (RunnableLambda(_format_to_messages) | model)})
add_routes(
    app,
    chat_model.with_types(input_type=ChatHistory),
    config_keys=["configurable"],
    path="/chat",
)


# Example 2: Chat Widget with History
# This one isn't hooked up toa model. It just shows that FunctionMessages can be used
# surfaced as well in the playground.


class ChatHistoryMessage(BaseModel):
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )
    location: str


def chat_message_bot(input: Dict[str, Any]) -> List[BaseMessage]:
    """Bot that repeats the question twice."""
    return [
        AIMessage(
            content="",
            additional_kwargs={
                "function_call": {
                    "name": "get_weather",
                    "arguments": dumps({"location": input["location"]}),
                }
            },
        ),
        FunctionMessage(name="get_weather", content='{"value": 32}'),
        AIMessage(content=f"Weather in {input['location']}: 32"),
    ]


add_routes(
    app,
    RunnableLambda(chat_message_bot).with_types(input_type=ChatHistoryMessage),
    config_keys=["configurable"],
    path="/chat_message",
)

# Example 3: File Processing Widget


class FileProcessingRequest(BaseModel):
    file: bytes = Field(..., extra={"widget": {"type": "base64file"}})
    num_chars: int = 100


def process_file(input: Dict[str, Any]) -> str:
    """Extract the text from the first page of the PDF."""
    content = base64.decodebytes(input["file"])
    blob = Blob(data=content)
    documents = list(PDFMinerParser().lazy_parse(blob))
    content = documents[0].page_content
    return content[: input["num_chars"]]


add_routes(
    app,
    RunnableLambda(process_file).with_types(input_type=FileProcessingRequest),
    config_keys=["configurable"],
    path="/pdf",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

ImportError: cannot import name 'field_validator' from 'pydantic' (c:\Users\xhx20\.conda\envs\Pytorch\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [3]:
!pip install --upgrade pydantic


  Obtaining dependency information for pydantic from https://files.pythonhosted.org/packages/d5/74/da832196702d0c56eb86b75bfa346db9238617e29b0b7ee3b8b4eccfe654/pydantic-2.10.2-py3-none-any.whl.metadata
     ------------------------------------ 170.8/170.8 kB 513.0 kB/s eta 0:00:00
  Obtaining dependency information for pydantic-core==2.27.1 from https://files.pythonhosted.org/packages/6a/fe/4e0e63c418c1c76e33974a05266e5633e879d4061f9533b1706a86f77d5b/pydantic_core-2.27.1-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.12.2 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ---------------------------------------- 456.4/456.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 6.3 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
 

ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'C:\\Users\\xhx20\\.conda\\envs\\Pytorch\\Lib\\site-packages\\~ydantic\\annotated_types.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

